# Linear Regression Model

## Loading and cleaning data

In [19]:
import pandas as pd

# 1. Load the CSV into a DataFrame
df = pd.read_csv("/Users/amelijaancupane/Desktop/TU_DELFT/Year 2/Advanced Data Analysis/NS_Project/Group2_NS-1/data/complete_dataset.csv", sep=";")


# 2. Data Cleaning
# Convert delay from string (HH:MM:SS) to numeric minutes
df["delay"] = pd.to_timedelta(df["delay"]).dt.total_seconds() / 60

# Drop rows where target is missing
df = df.dropna(subset=["REALISATIE"]).reset_index(drop=True)

# Fill categorical NaNs with "Unknown"
for col in ["DAGDEELTREIN", "TREINSERIEBASIS"]:
    df[col] = df[col].fillna("Unknown")

# Ensure TREINSERIEBASIS is stored as a string
df["TREINSERIEBASIS"] = df["TREINSERIEBASIS"].astype(str)

# Flag missing delay (NaN for cancelled trains)
df["delay_missing"] = df["delay"].isna().astype(int)

# Fill delay NaN with 0 so it's numeric
df["delay"] = df["delay"].fillna(0)

# Fill Disruption (minutes) NaN with 0 so it's numeric
df["Disruption (minutes)"] = df["Disruption (minutes)"].fillna(0)

# Fill in missing values for PROGNOSE_REIZEN (NaN if trains are extra)
df["prognose_missing"] = df["PROGNOSE_REIZEN"].isna().astype(int)
df["PROGNOSE_REIZEN"] = df["PROGNOSE_REIZEN"].fillna(0)

# Simplify weather columns to binary flags
df["Rain_flag"] = df["Rain"].notna().astype(int)
df["Heavy_Rain_flag"] = (df["Rain"] == "Heavy Rain").astype(int)
df["Gusts_flag"] = df["Gusts"].notna().astype(int)
df["Storms_flag"] = df["Storms"].notna().astype(int)

# (optional) drop the original text columns if you don’t need them anymore
# df = df.drop(columns=["Rain", "Gusts", "Storms"])


# 3. Inspect the data
print(df.head())          # first 5 rows
print(df.info())          # column info and data types
print(df.describe())      # summary statistics (numeric columns)



/var/folders/5z/nchhmd215419nr76bd71_jmw0000gn/T/ipykernel_11635/1272116139.py:4: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/amelijaancupane/Desktop/TU_DELFT/Year 2/Advanced Data Analysis/NS_Project/Group2_NS-1/data/complete_dataset.csv", sep=";")


   DAGNR  WEEK_DAG_NR   TRAJECT DAGDEELTREIN TREINSERIEBASIS PLANTIJD_VERTREK  \
0      1            6  Shl_Ledn      Weekend           700.0     05:57:00.000   
1      1            6  Ledn_Gvc      Weekend           700.0     06:15:00.000   
2      1            6  Shl_Ledn      Weekend           700.0     06:57:00.000   
3      1            6  Ledn_Gvc      Weekend           700.0     07:15:00.000   
4      1            6  Shl_Ledn      Weekend           700.0     07:57:00.000   

  UITVOERTIJD_AANKOMST  UITVOERTIJD_VERTREK  BEWEGINGNUMMER  REALISATIE  ...  \
0         06:16:12.000  2025-10-16 05:57:26             710   40.525088  ...   
1         06:28:32.000  2025-10-16 06:17:53             710   33.706422  ...   
2         07:17:18.000  2025-10-16 06:57:37             714   66.828816  ...   
3         07:29:32.000  2025-10-16 07:18:45             714   65.882354  ...   
4         08:17:35.000  2025-10-16 07:57:11             718  136.729115  ...   

   Disruption description  Rain 

In [20]:
def col_overview(df, col, topn=10):
    s = df[col]
    total = len(s)
    n_null = s.isna().sum()
    print(f"\n=== {col} ===")
    print(f"Total rows: {total:,}")
    print(f"NaNs: {n_null:,} ({n_null/total:.1%})")
    # Top non-null answers
    top = s.dropna().value_counts().head(topn)
    print(f"\nTop {topn} non-null values:")
    print(top)
col_overview(df, "Disruption description", topn=20)


=== Disruption description ===
Total rows: 98,617
NaNs: 98,366 (99.7%)

Top 20 non-null values:
Disruption description
HN: defect sp Sp-EA / Spoorweb-id: 991301                                                                                                                                                                                                                                                                                        9
suicidale vrouw wit shirt trainingsbroek                                                                                                                                                                                                                                                                                         9
suicidaal persoon te voorschoten                                                                                                                                                                                                          

## Set up model (library, target, features, split train-test set)

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [22]:
# Target
y = df["REALISATIE"]


# Features to use
features = ["WEEK_DAG_NR", "TRAJECT", "DAGDEELTREIN", "TREINSERIEBASIS",
            "PROGNOSE_REIZEN", "station1", "station2", 
            "Cancelled", "ExtraTrain", "delay", "delay_missing", "prognose_missing", "Rain_flag", "Heavy_Rain_flag", "Gusts_flag", "Storms_flag", "Disruption (minutes)"]

X = df[features]

# Convert categoricals with one-hot encoding
categorical_cols = ["TRAJECT", "DAGDEELTREIN", "TREINSERIEBASIS", "station1", "station2"]
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Chronological split by DAGNR - last 7 days as test set
cutoff_day = df["DAGNR"].max() - 7
X_train = X[df["DAGNR"] <= cutoff_day]
y_train = y[df["DAGNR"] <= cutoff_day]

X_test = X[df["DAGNR"] > cutoff_day]
y_test = y[df["DAGNR"] > cutoff_day]

print("Train:", X_train.shape, "Test:", X_test.shape)

Train: (75158, 138) Test: (23459, 138)


In [23]:
X.head()

,WEEK_DAG_NR,PROGNOSE_REIZEN,Cancelled,ExtraTrain,delay,delay_missing,prognose_missing,Rain_flag,Heavy_Rain_flag,Gusts_flag,...,station2_Hfd,station2_Laa,station2_Ledn,station2_Nvp,station2_Rsw,station2_Rtd,station2_Sdm,station2_Shl,station2_Ssh,station2_Vst
0,6,96.10463,False,False,0.000000,0,0,1,0,0,...,False,False,True,False,False,False,False,False,False,False
1,6,59.84863,False,False,2.883333,0,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
2,6,90.66623,False,False,0.000000,0,0,1,0,0,...,False,False,True,False,False,False,False,False,False,False
3,6,72.53823,False,False,3.750000,0,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
4,6,192.18303,False,False,0.000000,0,0,1,0,0,...,False,False,True,False,False,False,False,False,False,False


## Train and Test model

In [24]:
# --- Train Linear Regression ---
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

# --- Predict ---
y_pred = lin_model.predict(X_test)

# --- Evaluate ---
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Linear Regression Results on Test Set:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")
print("\n")

y_pred_baseline = X_test["PROGNOSE_REIZEN"]  # operator's estimate
mae_base = mean_absolute_error(y_test, y_pred_baseline)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
r2_base = r2_score(y_test, y_pred_baseline)

print(f"Operator Forecast (PROGNOSE_REIZEN) Results:")
print(f"MAE: {mae_base:.2f}")
print(f"RMSE: {rmse_base:.2f}")
print(f"R²: {r2_base:.3f}")


Linear Regression Results on Test Set:
MAE: 60.86
RMSE: 93.34
R²: 0.812


Operator Forecast (PROGNOSE_REIZEN) Results:
MAE: 61.69
RMSE: 99.62
R²: 0.786


## Try different feature combinations

In [27]:

# --- define feature sets to test ---
feature_sets = {
    "Set 1: Base + Delay": ["PROGNOSE_REIZEN", "prognose_missing", "delay", "delay_missing"],
    "Set 2: Base + Stations": ["PROGNOSE_REIZEN", "prognose_missing", "station1", "station2"],
    "Set 3: Delay + Stations": ["PROGNOSE_REIZEN", "prognose_missing", "delay", "delay_missing", "station1", "station2"],
    "Set 4: Add Route + Time": ["PROGNOSE_REIZEN", "prognose_missing", "delay", "delay_missing",
                                "station1", "station2", "TRAJECT", "DAGDEELTREIN"],
    "Set 5: Disruptions full": ["PROGNOSE_REIZEN", "prognose_missing", "delay", "delay_missing", "Cancelled", "ExtraTrain"],
    "Set 6: Full": ["WEEK_DAG_NR", "PROGNOSE_REIZEN", "prognose_missing", "delay", "delay_missing",
                    "station1", "station2", "TRAJECT", "DAGDEELTREIN", "TREINSERIEBASIS",
                    "Cancelled", "ExtraTrain"],
    "Set 7: Disruptions full + new": ["PROGNOSE_REIZEN", "prognose_missing", "delay", "delay_missing", "Cancelled", "ExtraTrain", "Disruption (minutes)", "Rain_flag", "Heavy_Rain_flag", "Gusts_flag", "Storms_flag"],
}

# --- results container ---
results = []

# --- split train/test by DAGNR (week 4 as test) ---
cutoff_day = df["DAGNR"].max() - 7
y = df["REALISATIE"]

for name, features in feature_sets.items():
    
    # copy so we don’t overwrite df
    X = df[features].copy()

    # one-hot encode categoricals
    cat_cols = [c for c in features if X[c].dtype == "object"]
    X = pd.get_dummies(X, columns=cat_cols, drop_first=True)
    
    # train/test split
    X_train = X[df["DAGNR"] <= cutoff_day]
    y_train = y[df["DAGNR"] <= cutoff_day]
    X_test  = X[df["DAGNR"] > cutoff_day]
    y_test  = y[df["DAGNR"] > cutoff_day]

    # operator forecast (baseline)
    y_pred_baseline = df.loc[df["DAGNR"] > cutoff_day, "PROGNOSE_REIZEN"]

    # fit Linear Regression
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    mae_base = mean_absolute_error(y_test, y_pred_baseline)
    rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
    r2_base = r2_score(y_test, y_pred_baseline)

    # --- loop over feature sets ---
for name, features in feature_sets.items():
    
    # copy so we don’t overwrite df
    X = df[features].copy()

    # one-hot encode categoricals
    cat_cols = [c for c in features if X[c].dtype == "object"]
    X = pd.get_dummies(X, columns=cat_cols, drop_first=True)
    
    # train/test split
    X_train = X[df["DAGNR"] <= cutoff_day]
    y_train = y[df["DAGNR"] <= cutoff_day]
    X_test  = X[df["DAGNR"] > cutoff_day]

    # fit Linear Regression
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Feature Set": name,
        "MAE (Model)": round(mae, 2),
        "RMSE (Model)": round(rmse, 2),
        "R² (Model)": round(r2, 3)
    })

# --- results table ---
results_df = pd.DataFrame(results)

# Add operator metrics as a single separate row
operator_row = pd.DataFrame([{
    "Feature Set": "Operator Forecast (baseline)",
    "MAE (Model)": round(mae_base, 2),
    "RMSE (Model)": round(rmse_base, 2),
    "R² (Model)": round(r2_base, 3)
}])

# Combine tables
results_df = pd.concat([operator_row, results_df], ignore_index=True)

print(results_df)

                     Feature Set  MAE (Model)  RMSE (Model)  R² (Model)
0   Operator Forecast (baseline)        61.69         99.62       0.786
1            Set 1: Base + Delay        60.25         94.92       0.806
2         Set 2: Base + Stations        59.97         94.68       0.807
3        Set 3: Delay + Stations        59.95         94.67       0.807
4        Set 4: Add Route + Time        59.78         93.52       0.812
5        Set 5: Disruptions full        60.25         94.92       0.806
6                    Set 6: Full        60.69         93.43       0.812
7  Set 7: Disruptions full + new        60.30         94.85       0.806


## Conclusion

Linear regression improves the predictions, but not very significantly. Will try more advanced ML models